In [61]:
import string
import pandas as pd

In [62]:
train_file = './data/sr_set-ud-train.conllu'
test_file = './data/sr_set-ud-test.conllu'
dev_file = './data/sr_set-ud-dev.conllu'

In [63]:
lines = []
with open(train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
with open(test_file, 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())
with open(dev_file, 'r', encoding='utf-8') as f:
    lines.extend(f.readlines())

In [64]:
open_class_tags = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'PROPN', 'VERB']

In [65]:
ud_content = []
for line in lines:
    if len(line) == 1:
        continue
    if line.startswith("#"):
        continue
    
    parts = line.split('\t') 
    word = parts[1]
    pos_tag = parts[3]
    if pos_tag not in open_class_tags:
        continue
    ud_content.append(word)

In [66]:
len(ud_content)

54632

# Preprocessing

All words are converted to lowercase.

In [67]:
ud_content = [word.lower() for word in ud_content]

Everything that contains digits and punctuation is removed.

In [68]:
ud_content = [word for word in ud_content if all(not c.isdigit() and c not in string.punctuation for c in word)]

In [69]:
data = pd.Series(ud_content)
data_freqs = data.value_counts()
data_freqs[:20]

rekao         332
godine        279
eu            261
takođe        166
bih           151
kada          141
kaže          132
zemlje        131
kosova        128
međutim       126
samo          120
evra          117
još           117
srbije        110
izjavio       107
trebalo       107
predsednik    106
odsto          99
više           98
ministar       94
dtype: int64

# Lexicon structure

Lexicon will be represented as a structure which consists of lemma (the base form), its tag and a list of the derived forms.
Each derived form consits of the form itself and its tag.

In [70]:
class LexiconEntry:
    def __init__(self, lemma, tag):
        self.lemma = lemma
        self.tag = tag
        self.derived_forms = []

In [71]:
lexicon = []

Defining useful functions:

In [72]:
def export_lexicon(filename):
    lines_to_export = []
    for lexicon_entry in lexicon:
        lemma = lexicon_entry.lemma
        tag = lexicon_entry.tag
        
        if len(lexicon_entry.derived_forms) == 0:
            line = "{} {}+{}".format(lemma, lemma, tag)
            lines_to_export.append(line)
        else:
            for derived_form_entry in lexicon_entry.derived_forms:
                derived_form, derived_form_info = derived_form_entry
                
                line = "{} {}+{}+{}".format(derived_form, lemma, tag, derived_form_info)
                lines_to_export.append(line)
    
    with open(filename, "w", encoding='utf-8') as f:
        f.write('\n'.join(lines_to_export))

In [73]:
def print_lexicon():
    for lexicon_entry in lexicon:
        lemma = lexicon_entry.lemma
        tag = lexicon_entry.tag

        print("{} [{}]".format(lemma, tag))
        if len(lexicon_entry.derived_forms) == 0:
            continue
        for idx, derived_form_entry in enumerate(lexicon_entry.derived_forms):
            derived_form, derived_form_info = derived_form_entry
            
            if idx == 0:
                print("    ∟ {} {}".format(derived_form, derived_form_info))
            else:
                print("      {} {}".format(derived_form, derived_form_info))

In [74]:
def add_to_lexicon(lemma, base_tag, derived_forms = []):
    global ud_content
    lexicon_entry = LexiconEntry(lemma, base_tag)
    for lex in lexicon:
        if lex.lemma == lemma and lex.tag == base_tag:
            return
    
    ud_content = [word for word in ud_content if word != lemma]

    for der_form in derived_forms:
        lexicon_entry.derived_forms.append(der_form)
    lexicon.append(lexicon_entry)
    
    derived_forms_words = [w[0] for w in derived_forms]
    ud_content = [word for word in ud_content if word not in derived_forms_words]

In [75]:
def get_lexicon_entry(lemma, tag):
    for lex_entry in lexicon:
        if lex_entry.lemma == lemma and lex_entry.tag == tag:
            return lex_entry
        
    return None

Let's start with the most distinctive suffixes, which can be easily be observed

# 1. Verbs

## 1.1 Infinitive

All verbs in Serbian end with: *-ti* or *-ći*

In [76]:
infinitive_verbs_ITI = list(set([w for w in ud_content if w.endswith('iti')]))

In [77]:
infinitive_verbs_ITI

['pripremiti',
 'izraditi',
 'oprostiti',
 'proširiti',
 'govoriti',
 'otkloniti',
 'unaprediti',
 'otplatiti',
 'pretpostaviti',
 'smanjiti',
 'pustiti',
 'značiti',
 'rešiti',
 'naslediti',
 'štiti',
 'zaštititi',
 'snositi',
 'razdvojiti',
 'ograničiti',
 'zavisiti',
 'patiti',
 'odstupiti',
 'zaključiti',
 'učiniti',
 'uložiti',
 'ponoviti',
 'raditi',
 'usredsrediti',
 'odrediti',
 'pojaviti',
 'obaviti',
 'prihvatiti',
 'potrošiti',
 'ponuditi',
 'preporučiti',
 'zameniti',
 'obezbediti',
 'izgubiti',
 'iznositi',
 'složiti',
 'odobriti',
 'omogućiti',
 'uočiti',
 'obilaziti',
 'ostaviti',
 'staviti',
 'izmeniti',
 'unakaziti',
 'razmotriti',
 'ohrabriti',
 'ispuniti',
 'odbaciti',
 'suočiti',
 'rijaliti',
 'osetiti',
 'tražiti',
 'vlastiti',
 'deliti',
 'zanemariti',
 'nastupiti',
 'vratiti',
 'udvostručiti',
 'izvršiti',
 'zatražiti',
 'nastaviti',
 'dozvoliti',
 'krediti',
 'razvodniti',
 'usuditi',
 'potvrditi',
 'pružiti',
 'shvatiti',
 'desiti',
 'protiviti',
 'ostvariti',


In [78]:
for inf_verb in infinitive_verbs_ITI:
    add_to_lexicon(inf_verb, 'V')

In [79]:
len(ud_content)

53283

In [80]:
infinitive_verbs_ETI = list(set([w for w in ud_content if w.endswith('eti')]))

In [81]:
infinitive_verbs_ETI

['oseti',
 'započeti',
 'preuzeti',
 'dopreti',
 'uzeti',
 'doprineti',
 'preduzeti',
 'doneti',
 'preoteti',
 'poseti',
 'preneti',
 'animoziteti',
 'anketi',
 'peti',
 'deveti',
 'našteti',
 'posveti',
 'tereti',
 'meti',
 'ćančeti',
 'ahmeti',
 'univerziteti',
 'uspeti',
 'antikviteti',
 'razumeti',
 'oteti',
 'početi',
 'dijeti',
 'oživeti',
 'preti',
 'videti',
 'predvideti',
 'podneti',
 'aveti',
 'predmeti',
 'fakulteti']

In [82]:
for inf_verb in infinitive_verbs_ETI:
    add_to_lexicon(inf_verb, 'V')

In [83]:
infinitive_verbs_ATI = list(set([w for w in ud_content if w.endswith('ati')]))

In [84]:
infinitive_verbs_ATI

['pozivati',
 'falsifikovati',
 'zahtevati',
 'blokirati',
 'dokazati',
 'kopirati',
 'uračunati',
 'apstinirati',
 'podržati',
 'poboljšati',
 'morati',
 'oklevati',
 'menjati',
 'omogućavati',
 'zadržati',
 'intervenisati',
 'garantovati',
 'stajati',
 'organizovati',
 'pozvati',
 'klati',
 'razmenjivati',
 'prati',
 'palati',
 'sastati',
 'pokazati',
 'pisati',
 'razrešavati',
 'primenjivati',
 'obraćati',
 'otputovati',
 'prisustvovati',
 'prebacivati',
 'sarađivati',
 'vrati',
 'investirati',
 'preispitati',
 'slati',
 'pokušati',
 'poštovati',
 'trajati',
 'poslati',
 'dati',
 'sećati',
 'povrati',
 'postupati',
 'održavati',
 'požurivati',
 'zarađivati',
 'prihvati',
 'plati',
 'zloupotrebljavati',
 'raspravljati',
 'prikazati',
 'uhvati',
 'obrati',
 'pridržavati',
 'rezultati',
 'zvati',
 'uticati',
 'postojati',
 'koštati',
 'funkcionisati',
 'pokušavati',
 'potpisati',
 'analizirati',
 'advokati',
 'posmatrati',
 'predstavljati',
 'rešavati',
 'dobijati',
 'modelirati',
 'ma

In [85]:
for inf_verb in infinitive_verbs_ATI:
    add_to_lexicon(inf_verb, 'V')

### 1.2 Past participle

Ending with: *-vši*

In [86]:
past_participle_VSI = list(set([w for w in ud_content if w.endswith('vši')]))
print(past_participle_VSI)

['počevši', 'uzdrmavši', 'učinivši', 'pretvorivši', 'osvojivši', 'proširivši', 'primetivši', 'dostigavši', 'postavivši', 'zamenivši', 'ponovivši', 'izazvavši', 'pohvalivši', 'bivši', 'završivši', 'rekavši', 'pobedivši', 'naslutivši', 'zabeleživši', 'dobivši']


In [87]:
for pp_VSI in past_participle_VSI:
    add_to_lexicon(pp_VSI, 'V')

### 1.3 Present participle

Ending with: *-jući*

In [88]:
def add_to_baseform(derived_form, derived_form_info, tag, src_replace, dst_replace):
    for lexicon_entry in lexicon:
        if lexicon_entry.lemma == derived_form.replace(src_replace, dst_replace) and lexicon_entry.tag == tag:
            lexicon_entry.derived_forms.append((derived_form, derived_form_info))
            return
    
    lemma = derived_form.replace(src_replace, dst_replace)
    derived_forms = [(derived_form, derived_form_info)]
    add_to_lexicon(lemma, tag, derived_forms)

In [89]:
present_participle_JUCI = list(set([w for w in ud_content if w.endswith('jući')]))

for pp in present_participle_JUCI:
    if pp.endswith('ajući'):
        add_to_baseform(pp, "PresentPat", "V", "ajući", "ati")

    elif pp.endswith('ujući'):
        add_to_baseform(pp, "PresentPat", "V", "ujući", "ovati")

In [90]:
present_participle_JUCI

['izražavajući',
 'postavljajući',
 'upozoravajući',
 'objašnjavajući',
 'završavajući',
 'ubacujući',
 'čitajući',
 'potvrđujući',
 'pozivajući',
 'ažurirajući',
 'optužujući',
 'odgovarajući',
 'pozdravljajući',
 'dodajući',
 'znajući',
 'otvarajući',
 'zahtevajući',
 'primajući',
 'narastajući',
 'predsedavajući',
 'insistirajući',
 'savetujući',
 'osuđujući',
 'imajući',
 'ohrabrujući',
 'okončavajući',
 'podsećajući',
 'obavljajući',
 'opisujući',
 'preduzimajući',
 'kritikujući',
 'napominjući',
 'uveravajući',
 'saopštavajući',
 'sprečavajući',
 'suočavajući',
 'oslobađajući',
 'čekajući',
 'izdvajajući',
 'diskreditujući',
 'zatvarajući',
 'protestujući',
 'odbijajući',
 'najavljujući',
 'nazivajući',
 'obraćajući',
 'emigrirajući',
 'zbližavajući',
 'prezajući',
 'zbunjujući',
 'ukazujući',
 'pretvarajući',
 'tragajući',
 'nedostajući',
 'prodajući',
 'primoravajući',
 'potiskujući',
 'uključujući',
 'bacajući',
 'igrajući',
 'zahvaljujući',
 'svrstavajući',
 'obeležavajući',


Another type of suffix of present participle form is *-eći*, but it can be mixed with infinitive form.

In [91]:
suffix_ECI = list(set([w for w in ud_content if w.endswith('eći')]))

In [92]:
suffix_ECI

['misleći',
 'odlazeći',
 'trudeći',
 'sledeći',
 'steći',
 'kršeći',
 'veći',
 'sudeći',
 'deleći',
 'donoseći',
 'nesreći',
 'beležeći',
 'preseći',
 'predstojeći',
 'preći',
 'koristeći',
 'važeći',
 'reći',
 'poreći',
 'doprinoseći',
 'tražeći',
 'postojeći',
 'treći',
 'navodeći',
 'paleći',
 'baveći',
 'izlazeći',
 'najveći',
 'govoreći',
 'tvrdeći']

# 2. Adjectives

## 2.1 Comparative

Suffixes:
- Singular: *-niji (m.), -nija (f.), -nije (n.)*
- Plural: *-nijim

In [93]:
comparative_forms = [w for w in ud_content if w.endswith('niji') or w.endswith('nija') or w.endswith('nije') or w.endswith('nijim')]
comparative_forms = [w for w in comparative_forms if not w.startswith('naj')]
comparative_forms = list(set(comparative_forms))

In [94]:
comparative_forms

['decenije',
 'unije',
 'važnija',
 'svestranije',
 'linija',
 'apolonija',
 'linije',
 'bezbedniji',
 'decenija',
 'kompleksnija',
 'moćniji',
 'menija',
 'pensilvaniji',
 'alianija',
 'liniji',
 'beskompromisniji',
 'ranije',
 'nezadovoljniji',
 'telefonije',
 'konkurentnija',
 'makedonije',
 'sigurnija',
 'snažnije',
 'efikasnija',
 'uticajniji',
 'unija',
 'ozbiljniji',
 'estonije',
 'pozitivniji',
 'čečeniji',
 'opasnijim',
 'kompanije',
 'španiji',
 'iskusnijim',
 'rasprostranjenija',
 'ceremonija',
 'litvanije',
 'značajniji',
 'kolonija',
 'britanije',
 'albaniji',
 'striktnije',
 'kompanija',
 'očigledniji',
 'potpunije',
 'britaniji',
 'docnije',
 'tačnije',
 'slovenije',
 'diversifikovanija',
 'ranijim',
 'kasnije',
 'izraženijim',
 'demokratičnija',
 'siromašnije',
 'britanija',
 'konkretnije',
 'jevtinijim',
 'španije',
 'precizniji',
 'ceremoniji',
 'kompaniji',
 'optimalnije',
 'sigurnije',
 'španija',
 'rumuniji',
 'rasprostranjeniji',
 'rumunije',
 'makedoniji',
 'zain

We will build positive forms of the comparative forms:

-niji $\rightarrow$ -an

-nija $\rightarrow$ -na

-nije $\rightarrow$ -no

-nijim $\rightarrow$ -an 
    - there's no way of knowing if the comparative form is supposed to be in masculine, feminine or neutral form, so here it is deliberately chosen that the masculine form will be the base form

In [95]:
for comparative in comparative_forms:
    positive_lexicon_form = None
    if comparative.endswith("niji"):
        positive = comparative.replace("niji", "an")
        positive_lexicon_form = (comparative, 'Comp+Masc+Sg')
    if comparative.endswith("nija"):
        positive = comparative.replace("nija", "na")
        positive_lexicon_form = (comparative, 'Comp+Fem+Sg')
    if comparative.endswith("nije"):
        positive = comparative.replace("nije", "no")
        positive_lexicon_form = (comparative, 'Comp+Neut+Sg')
    if comparative.endswith("nijim"):
        positive = comparative.replace("nijim", "an")
        positive_lexicon_form = (comparative, 'Comp+Masc+Pl')
    
    add_to_lexicon(positive, 'Adj', [positive_lexicon_form])

In [96]:
print_lexicon()

pripremiti [V]
izraditi [V]
oprostiti [V]
proširiti [V]
govoriti [V]
otkloniti [V]
unaprediti [V]
otplatiti [V]
pretpostaviti [V]
smanjiti [V]
pustiti [V]
značiti [V]
rešiti [V]
naslediti [V]
štiti [V]
zaštititi [V]
snositi [V]
razdvojiti [V]
ograničiti [V]
zavisiti [V]
patiti [V]
odstupiti [V]
zaključiti [V]
učiniti [V]
uložiti [V]
ponoviti [V]
raditi [V]
usredsrediti [V]
odrediti [V]
pojaviti [V]
obaviti [V]
prihvatiti [V]
potrošiti [V]
ponuditi [V]
preporučiti [V]
zameniti [V]
obezbediti [V]
izgubiti [V]
iznositi [V]
složiti [V]
odobriti [V]
omogućiti [V]
uočiti [V]
obilaziti [V]
ostaviti [V]
staviti [V]
izmeniti [V]
unakaziti [V]
razmotriti [V]
ohrabriti [V]
ispuniti [V]
odbaciti [V]
suočiti [V]
rijaliti [V]
osetiti [V]
tražiti [V]
vlastiti [V]
deliti [V]
zanemariti [V]
nastupiti [V]
vratiti [V]
udvostručiti [V]
izvršiti [V]
zatražiti [V]
nastaviti [V]
dozvoliti [V]
krediti [V]
razvodniti [V]
usuditi [V]
potvrditi [V]
pružiti [V]
shvatiti [V]
desiti [V]
protiviti [V]
ostvariti [V]


## 2.2 Superlative

It is fairly easy to find adjectives in superlative form because they almost always have the prefix *naj-*.

In [97]:
prefix_NAJ = list(set([w for w in ud_content if w.startswith('naj')]))

In [98]:
prefix_NAJ

['najefikasnije',
 'najduži',
 'najboljim',
 'najtiražnijeg',
 'najnovija',
 'najizolovanijih',
 'najslobodnija',
 'najvažnije',
 'najtraženija',
 'najmodernijeg',
 'najsigurnijih',
 'najavljena',
 'najznačajnijih',
 'najnoviju',
 'najtalentovanijih',
 'najtežih',
 'najznačajniji',
 'najbezbednijim',
 'najverovatniji',
 'najjačih',
 'najurgentniji',
 'najzad',
 'najboljem',
 'najbolja',
 'najbolji',
 'najznačajnija',
 'najstarijim',
 'najuglednijih',
 'najvažnija',
 'najnovijih',
 'najsiromašnijim',
 'najnasilniju',
 'najesen',
 'najprizemniji',
 'najpouzdniji',
 'najstarijih',
 'najozbiljniji',
 'najčuvenijih',
 'najneslobodnija',
 'najznačajnije',
 'najveću',
 'najbogatijih',
 'najvećim',
 'najraznovrsnije',
 'najviše',
 'najjednostavniji',
 'najveća',
 'najnovije',
 'najavljuje',
 'najvišeg',
 'najavila',
 'najprodavanije',
 'najmanju',
 'najgorem',
 'najmoćniji',
 'najveće',
 'najdelikatnijoj',
 'najvažnijih',
 'najrashlađenija',
 'najjače',
 'najnoviji',
 'najdinamičnija',
 'najva

In [99]:
def is_superlative_of_existing_base_form(lexicon_entry, superlative):
    if len(lexicon_entry.derived_forms) == 0:
        return False
    
    comparative = superlative[3:]
    for d_form in lexicon_entry.derived_forms:
        if d_form[1].startswith("Comp+") and d_form[0].startswith(comparative):
            gender = d_form[1].split("+")[1]
            number = d_form[1].split("+")[2]
            lexicon_entry.derived_forms.append((superlative, "Sup+{}+{}".format(gender, number)))
            return True
        
    return False
            

for p_NAJ in prefix_NAJ:
    lexicon_entries = [l for l in lexicon if is_superlative_of_existing_base_form(l, p_NAJ)]

In [100]:
print_lexicon()

pripremiti [V]
izraditi [V]
oprostiti [V]
proširiti [V]
govoriti [V]
otkloniti [V]
unaprediti [V]
otplatiti [V]
pretpostaviti [V]
smanjiti [V]
pustiti [V]
značiti [V]
rešiti [V]
naslediti [V]
štiti [V]
zaštititi [V]
snositi [V]
razdvojiti [V]
ograničiti [V]
zavisiti [V]
patiti [V]
odstupiti [V]
zaključiti [V]
učiniti [V]
uložiti [V]
ponoviti [V]
raditi [V]
usredsrediti [V]
odrediti [V]
pojaviti [V]
obaviti [V]
prihvatiti [V]
potrošiti [V]
ponuditi [V]
preporučiti [V]
zameniti [V]
obezbediti [V]
izgubiti [V]
iznositi [V]
složiti [V]
odobriti [V]
omogućiti [V]
uočiti [V]
obilaziti [V]
ostaviti [V]
staviti [V]
izmeniti [V]
unakaziti [V]
razmotriti [V]
ohrabriti [V]
ispuniti [V]
odbaciti [V]
suočiti [V]
rijaliti [V]
osetiti [V]
tražiti [V]
vlastiti [V]
deliti [V]
zanemariti [V]
nastupiti [V]
vratiti [V]
udvostručiti [V]
izvršiti [V]
zatražiti [V]
nastaviti [V]
dozvoliti [V]
krediti [V]
razvodniti [V]
usuditi [V]
potvrditi [V]
pružiti [V]
shvatiti [V]
desiti [V]
protiviti [V]
ostvariti [V]


- Prisvojni pridevi (mesta): -ski, -čki, -ški
- Prisvojni pridevi (?): -ni
- Prisvojni pridevi (ljudi): -ov, -in, -ev

## 2.3 Possessive adjectives

Possessive adjectives can be of the following forms depending on the type of entity that *possesses*:

1. *-ov, -ev, -in*: when the entity that possesses is a human being
    - Examples: Markov "Mark's", Tonijev "Tony's", Nemanjin "Nemanja's" 
2. *-ski, -čki, -ški*: usually with toponymical entities - countries, ethnicities, cities, villages, regions, etc.
    - Examples: srpski "Serbian", nemački "German", praški "Prague's"
3. *-ni*
    - Examples: državni "state"

Of course, adjectives are changed according to gender as well, so we have:

2. *-ska* (f.), *-sko* (n.), *-čka* (f.), *-čko (n.), *-ška* (f.), *-ško* (n.)
3. *-na* (f.), *-no* (n.)

However, the inflection forms for masculine, feminine and neutral gender are the same when inflecting over different cases. The only difference is in the nominative form, so feminine and neutral forms are put under the masculine form.

Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|srpski|srpska|srpsko|
|genitive|srpskog|srpske|srpskog|
|dative|srpskom|srpskoj|srpskom|
|acusative|srpski|srpsku|srpsko|
|vocative|srpski|srpska|srpsko|
|instrumental|srpskim|srpskom|srpskim|
|locative|srpskom|srpskoj|srpskom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|srpski|srpske|srpska|
|genitive|srpskih|srpskih|srpske|
|dative|srpskim|srpskim|srpskoj|
|acusative|srpske|srpske|srpsku|
|vocative|srpski|srpske|srpska|
|instrumental|srpskim|srpskim|srpskom|
|locative|srpskim|srpskim|srpskoj|

Possible siffixes: *-ski*, *-skog*, *-skom*, *-skim*, *-ska*, *-ske*, *-skoj*, *-sku*, *-skih*, *-sko*

### 2.3.1 suffix *-ski* and its inflections

In [101]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('ski')]))

In [102]:
possessive_adjectives

['finansijski',
 'trojanski',
 'dostojevski',
 'svetski',
 'herkulovski',
 'romski',
 'ivanovski',
 'generacijski',
 'mađarski',
 'beogradski',
 'istorijski',
 'gruevski',
 'britanski',
 'diplomatski',
 'kurdski',
 'filmski',
 'švajcarski',
 'televizijski',
 'spoljnotrgovinski',
 'spisateljski',
 'pakistanski',
 'socijaldemokratski',
 'bosforski',
 'dejtonski',
 'balkanski',
 'irski',
 'geografski',
 'vremenski',
 'albanski',
 'belgijski',
 'bosanski',
 'građevinski',
 'londonski',
 'lisabonski',
 'teoretski',
 'jugoslovenski',
 'terapeutski',
 'portugalski',
 'vijetnamski',
 'cerovski',
 'tetovski',
 'carinski',
 'avionski',
 'crvenkovski',
 'saudijski',
 'industrijski',
 'zemunski',
 'olimpijski',
 'posttraumatski',
 'mančevski',
 'nikolovski',
 'orvelovski',
 'argentinski',
 'imunski',
 'većinski',
 'puninski',
 'engleski',
 'turski',
 'trajkovski',
 'kablovski',
 'bučkovski',
 'moldavski',
 'poljski',
 'italijanski',
 'rumunski',
 'proevropski',
 'amaterski',
 'antonovski',
 'franc

In [103]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "Adj")

In [104]:
def add_possessive_adjectives(possessive_adjectives, replace_src, replace_dst):
    for possessive_ADJ in possessive_adjectives:
        derived_f = (possessive_ADJ, "Poss")
        base_form = possessive_ADJ.replace(replace_src, replace_dst)
        lexicon_entry = get_lexicon_entry(base_form, "Adj")
        if lexicon_entry is not None:
            lexicon_entry.derived_forms.append(derived_f)
        else:
            add_to_lexicon(base_form, "Adj", [derived_f])

In [105]:
def add_adjective_inflection_forms(inflection_suffixes):
    for suffix, base_suffix in inflection_suffixes:
        possessive_adjectives = list(set([w for w in ud_content if w.endswith(suffix)]))
        add_possessive_adjectives(possessive_adjectives, suffix, base_suffix)

In [106]:
inflection_suffixes = [('skog', 'ski'), ('skom', 'ski'), ('skim', 'ski'), ('ska', 'ski'), ('ske', 'ski'), 
                       ('skoj', 'ski'), ('sku', 'ski'), ('skih', 'ski'), ('sko', 'ski')]
add_adjective_inflection_forms(inflection_suffixes)

In [107]:
print_lexicon()

pripremiti [V]
izraditi [V]
oprostiti [V]
proširiti [V]
govoriti [V]
otkloniti [V]
unaprediti [V]
otplatiti [V]
pretpostaviti [V]
smanjiti [V]
pustiti [V]
značiti [V]
rešiti [V]
naslediti [V]
štiti [V]
zaštititi [V]
snositi [V]
razdvojiti [V]
ograničiti [V]
zavisiti [V]
patiti [V]
odstupiti [V]
zaključiti [V]
učiniti [V]
uložiti [V]
ponoviti [V]
raditi [V]
usredsrediti [V]
odrediti [V]
pojaviti [V]
obaviti [V]
prihvatiti [V]
potrošiti [V]
ponuditi [V]
preporučiti [V]
zameniti [V]
obezbediti [V]
izgubiti [V]
iznositi [V]
složiti [V]
odobriti [V]
omogućiti [V]
uočiti [V]
obilaziti [V]
ostaviti [V]
staviti [V]
izmeniti [V]
unakaziti [V]
razmotriti [V]
ohrabriti [V]
ispuniti [V]
odbaciti [V]
suočiti [V]
rijaliti [V]
osetiti [V]
tražiti [V]
vlastiti [V]
deliti [V]
zanemariti [V]
nastupiti [V]
vratiti [V]
udvostručiti [V]
izvršiti [V]
zatražiti [V]
nastaviti [V]
dozvoliti [V]
krediti [V]
razvodniti [V]
usuditi [V]
potvrditi [V]
pružiti [V]
shvatiti [V]
desiti [V]
protiviti [V]
ostvariti [V]


### 2.3.2 suffix *-čki* and its inflections


Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|nemački|nemačka|nemačko|
|genitive|nemačkog|nemačke|nemačkog|
|dative|nemačkom|nemačkoj|nemačkom|
|acusative|nemački|nemačku|nemačko|
|vocative|nemački|nemačka|nemačko|
|instrumental|nemačkim|nemačkom|nemačkim|
|locative|nemačkom|nemačkoj|nemačkom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|nemački|nemačke|nemačka|
|genitive|nemačkih|nemačkih|nemačke|
|dative|nemačkim|nemačkim|nemačkoj|
|acusative|nemačke|nemačke|nemačku|
|vocative|nemački|nemačke|nemačka|
|instrumental|nemačkim|nemačkim|nemačkom|
|locative|nemačkim|nemačkim|nemačkoj|

Possible suffixes: *-čki*, *-čkog*, *-čkom*, *-čka*, *-čkih*, *-čkim*, *-čko*, *-čke*, *-čkoj*, *-čku*

In [108]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('čki')]))

In [109]:
possessive_adjectives

['populistički',
 'nemački',
 'železnički',
 'višestranački',
 'etnički',
 'muzički',
 'posmatrački',
 'pacifički',
 'stranački',
 'ultranacionalistički',
 'slovenački',
 'brodovlasnički',
 'irački',
 'predsednički',
 'turistički',
 'prostački',
 'politički',
 'zajednički',
 'umetnički',
 'putnički',
 'monopolistički',
 'komunistički',
 'statistički',
 'preduzetnički',
 'tehnički',
 'međuetnički',
 'radnički',
 'grčki',
 'bošnjački',
 'ekstremistički',
 'teroristički',
 'američki',
 'pregovarački']

In [110]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "Adj")

In [111]:
inflection_suffixes = [('čkog', 'čki'), ('čkom', 'čki'), ('čka', 'čki'), ('čkih', 'čki'), ('čkim', 'čki'), ('čko', 'čki'), 
                      ('čke', 'čki'), ('čkoj', 'čki'), ('čku', 'čki')]

add_adjective_inflection_forms(inflection_suffixes)

In [112]:
print_lexicon()

pripremiti [V]
izraditi [V]
oprostiti [V]
proširiti [V]
govoriti [V]
otkloniti [V]
unaprediti [V]
otplatiti [V]
pretpostaviti [V]
smanjiti [V]
pustiti [V]
značiti [V]
rešiti [V]
naslediti [V]
štiti [V]
zaštititi [V]
snositi [V]
razdvojiti [V]
ograničiti [V]
zavisiti [V]
patiti [V]
odstupiti [V]
zaključiti [V]
učiniti [V]
uložiti [V]
ponoviti [V]
raditi [V]
usredsrediti [V]
odrediti [V]
pojaviti [V]
obaviti [V]
prihvatiti [V]
potrošiti [V]
ponuditi [V]
preporučiti [V]
zameniti [V]
obezbediti [V]
izgubiti [V]
iznositi [V]
složiti [V]
odobriti [V]
omogućiti [V]
uočiti [V]
obilaziti [V]
ostaviti [V]
staviti [V]
izmeniti [V]
unakaziti [V]
razmotriti [V]
ohrabriti [V]
ispuniti [V]
odbaciti [V]
suočiti [V]
rijaliti [V]
osetiti [V]
tražiti [V]
vlastiti [V]
deliti [V]
zanemariti [V]
nastupiti [V]
vratiti [V]
udvostručiti [V]
izvršiti [V]
zatražiti [V]
nastaviti [V]
dozvoliti [V]
krediti [V]
razvodniti [V]
usuditi [V]
potvrditi [V]
pružiti [V]
shvatiti [V]
desiti [V]
protiviti [V]
ostvariti [V]


### 2.3.3 suffix *-ški* and its inflections


Singular

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|češki|češka|češko|
|genitive|češkog|češke|češkog|
|dative|češkom|češkoj|češkom|
|acusative|češki|češku|češko|
|vocative|češki|češka|češko|
|instrumental|češkim|češkom|češkim|
|locative|češkom|češkoj|češkom|

Plural

| case | masculine | feminine | neutral | 
|------|-----------|----------|---------|
|nominative|češki|češke|češka|
|genitive|čeških|čeških|češke|
|dative|češkim|češkim|češkoj|
|acusative|češke|češke|češku|
|vocative|češki|češke|češka|
|instrumental|češkim|češkim|češkom|
|locative|češkim|češkim|češkoj|

Possible suffixes: *-ški*, *-škog*, *-škom*, *-ška*, *-ških*, *-škim*, *-ško*, *-ške*, *-škoj*, *-šku*

In [113]:
possessive_adjectives = list(set([w for w in ud_content if w.endswith('ški')]))

In [114]:
possessive_adjectives

['češki',
 'muški',
 'tehnološki',
 'ideološki',
 'njujorški',
 'bombaški',
 'malteški',
 'norveški',
 'strateški',
 'haški',
 'zoološki',
 'luksemburški']

In [115]:
for possesive_adj in possessive_adjectives:
    add_to_lexicon(possesive_adj, "Adj")

In [116]:
inflection_suffixes = [('škog', 'ški'), ('škom', 'ški'), ('ška', 'ški'), 
                       ('ških', 'ški'), ('škim', 'ški'), ('ško', 'ški'), 
                       ('ške', 'ški'), ('škoj', 'ški'), ('šku', 'ški')]

add_adjective_inflection_forms(inflection_suffixes)

In [117]:
print_lexicon()

pripremiti [V]
izraditi [V]
oprostiti [V]
proširiti [V]
govoriti [V]
otkloniti [V]
unaprediti [V]
otplatiti [V]
pretpostaviti [V]
smanjiti [V]
pustiti [V]
značiti [V]
rešiti [V]
naslediti [V]
štiti [V]
zaštititi [V]
snositi [V]
razdvojiti [V]
ograničiti [V]
zavisiti [V]
patiti [V]
odstupiti [V]
zaključiti [V]
učiniti [V]
uložiti [V]
ponoviti [V]
raditi [V]
usredsrediti [V]
odrediti [V]
pojaviti [V]
obaviti [V]
prihvatiti [V]
potrošiti [V]
ponuditi [V]
preporučiti [V]
zameniti [V]
obezbediti [V]
izgubiti [V]
iznositi [V]
složiti [V]
odobriti [V]
omogućiti [V]
uočiti [V]
obilaziti [V]
ostaviti [V]
staviti [V]
izmeniti [V]
unakaziti [V]
razmotriti [V]
ohrabriti [V]
ispuniti [V]
odbaciti [V]
suočiti [V]
rijaliti [V]
osetiti [V]
tražiti [V]
vlastiti [V]
deliti [V]
zanemariti [V]
nastupiti [V]
vratiti [V]
udvostručiti [V]
izvršiti [V]
zatražiti [V]
nastaviti [V]
dozvoliti [V]
krediti [V]
razvodniti [V]
usuditi [V]
potvrditi [V]
pružiti [V]
shvatiti [V]
desiti [V]
protiviti [V]
ostvariti [V]


In [118]:
export_lexicon("lexicon.txt")